In [17]:
import mlflow
import dotenv

In [18]:
mlflow.set_tracking_uri('http://localhost:5000/')
mlflow.set_registry_uri('http://localhost:5000/')

In [19]:
from sklearn.linear_model import LinearRegression

In [21]:
mlflow.set_experiment('t')
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(sk_model=LinearRegression(),artifact_path='model')

In [ ]:
import boto

In [5]:
!pip install minio

     |████████████████████████████████| 81kB 2.6MB/s eta 0:00:01


In [5]:
from minio import Minio
import json
import os
import dotenv

In [6]:
dotenv.load_dotenv('.env')

True

In [7]:
os.environ['MLFLOW_S3_ENDPOINT_URL']

'http://localhost:9000'

In [10]:
minioClient = Minio(os.environ['MLFLOW_S3_ENDPOINT_URL'].split('//')[1],
                  access_key=os.environ['MINIO_ACCESS_KEY'],
                  secret_key=os.environ['MINIO_SECRET_KEY'],
                  secure=False)

In [ ]:
minioClient = Minio(os.environ['MLFLOW_S3_ENDPOINT_URL'].split('//')[1],
                  access_key=os.environ['AWS_ACCESS_KEY_ID'],
                  secret_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                  secure=False)

In [11]:
minioClient.list_buckets()

MaxRetryError: HTTPSConnectionPool(host='localhost', port=9000): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_record', 'wrong version number')])")))

In [25]:
try:
    minioClient.make_bucket('mlflow')
except Exception as err:
    print(err)

In [46]:
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mlflow 2021-04-07 00:46:56.603000+00:00


In [33]:
policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflow"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::mlflow"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflow/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflow/*"
            }

        ]}

In [44]:
minioClient.set_bucket_policy('mlflow', json.dumps(policy))

In [29]:
# List all object paths in bucket that begin with my-prefixname.
objects = minioClient.list_objects('mlflow',recursive=True)
for obj in objects:
    print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
          obj.etag, obj.size, obj.content_type)